In [6]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import pandas as pd
import numpy as np
import cv2

In [7]:
columns=[]
data=[]
headers={
'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64'
}


for page in range(1,100,100):
    url='https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/'+str(page)
    response=requests.get(url,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    table_html=soup.find('table')
    
    for th in table_html.find_all('th'):
        if th.text not in columns:
            columns.append(th.text.strip())

    for tr in table_html.find_all('tr'):
        row=[]

        for td in tr.find_all('td'):
            row.append(td.text.strip())

        all_a_tags=tr.find_all('a')
        if all_a_tags:
            link_movie=all_a_tags[-1].get('href')

            url_movie='https://www.the-numbers.com' + link_movie
            response_movie=requests.get(url_movie,headers=headers)
            soup_movie=BeautifulSoup(response_movie.content,'html.parser')
            summary=soup_movie.find(id='summary')
            for title in summary.find_all('h2'):
                if(title.text=='Movie Details'):
                    detail_table=title.next_sibling.next_sibling
                    for tr_detail_table in detail_table.find_all('tr'):
                        th=tr_detail_table.find_all('td')[0]
                        td=tr_detail_table.find_all('td')[1]
                        if(th.text=='Running Time:' or th.text=='Genre:' or th.text=='Creative Type:' or th.text=='Production/Financing Companies:' or th.text=='Production Countries:' or th.text=='Languages:'):
                            if th.text not in columns:
                                columns.append(th.text)
                            row.append(td.text)
                    break

        if row:
            data.append(row)


KeyboardInterrupt: 

In [9]:

df = pd.DataFrame(data, columns=columns)
df.to_csv('SmallDS_raw.csv')
# mssv_list=df['Số thẻ SV']

# for mssv in mssv_list:
#     img_resp = requests.get('http://cb.dut.udn.vn/ImageSV/20/'+mssv+'.jpg')
#     if(img_resp):
#         with open(mssv+'.jpg', 'wb') as f:
#             f.write(img_resp.content)
